# Song Classification, Part 2

## This exercise comes from the Berkeley data science teaching materials

This exercise is a version of lab 5 from section 3,  from
<https://github.com/data-8/materials-x19> with thanks.  See that URL for more
on the license for these materials.  We release this version, like the
original, under the  [Creative Commons Attribution-NonCommercial 4.0
International license](https://creativecommons.org/licenses/by-nc/4.0) (CC
BY-NC 4.0).

## Welcome

Welcome to part 2 of the song classifier! We'll pick off where we left off and
continue to build a song classifier using k-nearest neighbors.

Please complete part 1 before starting part 2.

You will build a classifier that guesses whether a song is hip-hop or country,
using only the number of times that each word appears in a song's lyrics.  By
the end of the project, you should know how to:

1. Clean and organize a dataset used to test a machine learning model
2. Build a k-nearest neighbors classifier
3. Test a classifier on data

**Advice.** Develop your answers incrementally. To perform a complicated table
manipulation, break it up into steps, perform each step on a different line,
give a new name to each result, and check that each intermediate result is what
you expect. You can add any additional names or functions you want to the
provided cells.

To get started, load our usual libraries.

In [ ]:
# Run this cell to set up the notebook, but please don't change it.
import numpy as np
import pandas as pd
# Safe settings for Pandas.
pd.set_option('mode.chained_assignment', 'raise')

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Load the OKpy test library and tests.
from client.api.notebook import Notebook
ok = Notebook('classify_song_2.ok')

## Overview: Recap

In part 1, we completed the following tasks:

1. In section 1, we explored the dataset and split the dataset into training
   data and test data.
2. In section 2, we ran through a guided example of the k-Nearest Neighbors
   (k-NN) classification algorithm.

**If you do not remember part 1, we highly recommend you go back and review it
now. It will help you for this exercise.**

In this part - part 1, we are going to complete the following tasks:

1. Identify some features.
2. Define a classifier function using your features and the training set.
3. Evaluate its performance (the proportion of correct classifications) on the
   test set.

Run the cell below to set up the lab.

In [ ]:
# Run some processing from part 1 of the exercise.
lyrics = pd.read_csv('lyrics.csv')
# Title as index
lyrics = lyrics.set_index('Title', drop=False)

training_proportion = 11/16

num_songs = len(lyrics)
num_train = int(num_songs * training_proportion)
num_valid = num_songs - num_train

train_lyrics = lyrics.iloc[:num_train]
test_lyrics = lyrics.iloc[num_train:]

def most_common(col_name, df):
    """The most common element in a column of a data frame.

    Parameters
    ----------
    col_name : str
        A string; the name of a column.
    df : DataFrame

    Returns
    -------
    commonest : value
        The most common value in that column of that table. In case of a tie,
        it returns any one of the most common values.
    """
    counts = df[col_name].value_counts()
    return counts.index[0]

## 1. Features

Now, we're going to extend our classifier from  to consider more than two
features at a time.

Euclidean distance still makes sense with more than two features. For `n`
different features, we compute the difference between corresponding feature
values for two songs, square each of the `n`  differences, sum up the resulting
numbers, and take the square root of the sum.


### Question 1.1

Write a function to compute the Euclidean distance between two **arrays** of
features of *arbitrary* (but equal) length.  Use it to compute the distance
between the first song in the training set and the first song in the test set,
*using all of the features*.  (Remember that the title, artist, and genre of
the songs are not features.)

*Hint:* To convert rows to arrays, use `np.array`. For example, if `t` was a
data frame, `np.array(t.iloc[0])` converts row 0 of `t` into an array.

*Hint:* Remember Pandas `iloc` indexing to get data frame data by position
(rather than by label).  For example, to get the row at position 9, and all the
columns from position 5, you could use `t.iloc[9, 5:]`.

In [ ]:
def distance(features1, features2):
    """The Euclidean distance between two arrays of feature values."""
    ...

distance_first_to_first = ...
distance_first_to_first

In [ ]:
_ = ok.grade("q1_1")

### 1.1. Creating your own feature set

Unfortunately, using all of the features has some downsides.  One clear
downside is *computational* -- computing Euclidean distances just takes a long
time when we have lots of features.  You might have noticed that in the last
question!

So we're going to select just 20.  We'd like to choose features that are very
*discriminative*. That is, features which lead us to correctly classify as much
of the test set as possible.  This process of choosing features that will make
a classifier work well is sometimes called *feature selection*, or more broadly
*feature engineering*.


### Question 1.1.1

Look through the list of features (the column names of the `lyrics` data frame
after the first three).  Choose 20 common words that you think might let you
distinguish between country and hip-hop songs. Make sure to choose words that
are frequent enough that every song contains at least one of them. Don't just
choose the 20 most frequent, though... you can do much better.

You might want to come back to this question later to improve your list, once
you've seen how to evaluate your classifier.  The first time you answer this
question, spend some time looking through the features, but not more than 15
minutes.

In [ ]:
#- Set my_20_features to a list of 20 features (strings that are column labels)
my_20_features = ...

# Select columns using loc (label) indexing.
train_20 = train_lyrics.loc[:, my_20_features]
test_20 = test_lyrics.loc[:, my_20_features]

This test below makes sure that you have chosen words such that at least one
appears in each song. If you can't find words that satisfy this test just
through intuition, try writing code to print out the titles of songs that do
not contain any words from your list, then look at the words they do contain.

In [ ]:
_ = ok.grade("q1_1_1")

Next, let's classify the first song from our test set using these features.
You can examine the song by running the cells below. Do you think it will be
classified correctly?

In [ ]:
print("Song:")
print(test_lyrics.iloc[0, :3])
print("Features:")
test_20.iloc[0]

As before, we want to look for the songs in the training set that are most like
our test song, in terms of our features.  We will calculate the Euclidean
distances from the test song (using the 20 selected features) to all songs in
the training set.  You could do this with a `for` loop, but to make it
computationally faster, we have provided a function, `fast_distances`, to do
this for you.  Read its documentation to make sure you understand what it does.
(You don't need to read the code in its body unless you want to.)

In [ ]:
# Just run this cell to define fast_distances.

def fast_distances(test_row, train_rows):
    """Array of distances between `test_row` and each row in `train_rows`.

    Parameters
    ----------
    test_row: attribute series / array
        A row of a table containing features of one test song (e.g.,
        test_20.iloc[0]).
    train_rows: data frame
        A table of features (for example, the whole table train_20).

    Returns
    -------
    distances : array
        One distance per row in `train_rows`.
    """
    assert train_rows.shape[1] < 50, "Make sure you're not using all the features of the lyrics table."
    # Convert the test row to an array of floating point values.
    test_row = np.array(test_row).astype(np.float64)
    # Convert the training attributes data frame to an array of floating point
    # values.
    train_attrs = np.array(train_rows).astype(np.float64)
    # Make an array of the same shape as `train_attrs` by repeating the
    # `test_row` len(train_rows) times.
    repeated_test_row = np.tile(test_row, [len(train_rows), 1])
    # Now we can do the subtractions all at once.
    diff = repeated_test_row - train_attrs
    distances = np.sqrt(np.sum(diff ** 2, axis=1))
    return distances

### Question 1.1.2

Use the `fast_distances` function provided above to compute the distance from
the first song in the test set to all the songs in the training set, **using
your set of 20 features**.  Make a new table called `genre_and_distances` with
one row for each song in the training set and two columns:

* The `"Genre"` of the training song
* The `"Distance"` from the first song in the test set 

Ensure that `genre_and_distances` is **sorted in increasing order by distance
to the first test song**.

In [ ]:
#- The staff solution took 4 lines of code.
genre_and_distances = ...
genre_and_distances

In [ ]:
_ = ok.grade("q1_1_2")

### Question 1.1.3

Now compute the 5-nearest neighbors classification of the first song in the
test set.  That is, decide on its genre by finding the most common genre among
its 5 nearest neighbors, according to the distances you've calculated.  Then
check whether your classifier chose the right genre.  (Depending on the
features you chose, your classifier might not get this song right, and that's
okay.)

*Hint*: don't forget the `most_common` function you wrote for part 1, and
replicated at the top here for your use.

In [ ]:
#- Set my_assigned_genre to the most common genre among these.
my_assigned_genre = ...
print(my_assigned_genre)

#- Set my_assigned_genre_was_correct to True if my_assigned_genre
#- matches the actual genre of the first song in the test set.
my_assigned_genre_was_correct = ...

print("The assigned genre, {}, was{}correct.".format(my_assigned_genre, " " if my_assigned_genre_was_correct else " not "))

In [ ]:
_ = ok.grade("q1_1_3")

### 1.2. A classifier function

Now we can write a single function that encapsulates the whole process of
classification.


### Question 1.2.1

Write a function called `classify`.  It should take the following four
arguments:

* A row of features for a song to classify (e.g., `test_20.iloc[0]`).
* A data frame with a column for each feature (e.g., `train_20`).
* An array of classes that has as many items as the previous table has rows,
  and in the same order.  These are the "Country" and "Hip-hop" labels for each
  of the training songs.
* `k`, the number of neighbors to use in classification.

It should return the class that a `k`-nearest neighbor classifier picks for the
given row of features (the string `'Country'` or the string `'Hip-hop'`).

In [ ]:
def classify(test_row, train_rows, train_classes, k):
    """Return the most common class among k nearest neighbors to test_row.

    Parameters
    ----------
    test_row : series / array
        Attributes for a single row (in our case, attributes for a single
        song).
    train_rows : data frame
        Data frame containing attributes in training set, one row per item in the training set (in our case - one row per song).
    train_classes : series / array / list
        Sequence containing strings, where each element is the label
        corresponding to the equivalent row in the `train_rows` data frame.  In our case these will be the 'Country' and 'Hip-hop' labels for each song in the `train_rows` data frame.
    k : int
        Number of nearest neighbors to consider.

    Returns
    -------
    label : str
        Predicted label for `test_row`.
    """
    distances = fast_distances(test_row, train_rows)
    genre_and_distances = ...
    ...

In [ ]:
_ = ok.grade("q1_2_1")

## Question 1.2.2

Assign `grandpa_genre` to the genre predicted by your classifier for the song
"Grandpa Got Runned Over By A John Deere" in the test set, using **9
neighbors** and using your 20 features.

*Hint*: remember, because we set the `lyrics` index to be the song title, you
can use the song title when indexing with `.loc`.

In [ ]:
#- The staff solution first defined a row object called grandpa_features.
grandpa_features = ...
grandpa_genre = ...
grandpa_genre

In [ ]:
_ = ok.grade("q1_2_2")

Finally, when we evaluate our classifier, it will be useful to have a
classification function that is specialized to use a fixed training set and a
fixed value of `k`.


### Question 1.2.3

Create a classification function that takes as its argument a row containing your 20 features and classifies that row using the 5-nearest neighbors algorithm with `train_20` as its training set.

In [ ]:
def classify_one_argument(row):
    """ Classify `row` using `train_20` training set, 5 nearest neighbors

    Parameters
    ----------
    row : Series / array
        Attributes for a single row.

    Returns
    -------
    label : str
        Predicted label for song with attributes in `row`.
    """
    ...

# When you're done, this should produce 'Hip-hop' or 'Country'.
classify_one_argument(test_20.iloc[0])

In [ ]:
_ = ok.grade("q1_2_3")

### 1.3. Evaluating your classifier

Now that it's easy to use the classifier, let's see how accurate it is on the whole test set.

**Question 1.3.1.** Use `classify_one_argument` and `apply` to classify every song in the test set.  Name these guesses `test_guesses`.  **Then**, compute the proportion of correct classifications. 

In [ ]:
test_guesses = ...
proportion_correct = ...
proportion_correct

In [ ]:
_ = ok.grade("q1_3_1")

At this point, you've gone through one cycle of classifier design.  Let's summarize the steps:

1. From available data, select test and training sets.
2. Choose an algorithm you're going to use for classification.
3. Identify some features.
4. Define a classifier function using your features and the training set.
5. Evaluate its performance (the proportion of correct classifications) on the test set.


## The end

Congratulations! You're finished with part 2! You've created your own song
classifier using k-nearest neighbors.

**If you want to continue, you can read about classification online and try to
build an even more accurate classifier.**

Be sure to:

- **run all the tests** (the next cell has a shortcut for that).
- **Save and Checkpoint** from the `File` menu.

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]